# Adaptive Circuit Constrction

## Introduction

Adaptive circuit construction offers a way to build up optimal circuits for VQE. In those algorithms, ansatz circuits are adaptively constructed by selecting and adding blocks from a predefined pool $\mathcal{E}$.

Specifically, in each iteration, we do the following steps
- Go over the blocks in $\mathcal{E}$;
- Calculate a score of every block based on certain criterion;
- Add the entangler with highest score to the circuit.

We believe circuit construction is important for *near-term* quantum computing because their adaptiveness can help them place the entangling gates in a more efficient way than the methods use fixed circuits. The two-qubit gate advantage of including adaptiveness has been shown in some works.

To design a adaptive circuit construction algorithm, one needs to define
- The blocks pool $\mathcal{E}$;
- The criterion for scoring the blocks;
- Where to add the new block.

In Mizore, we provide a general framework `CircuitConstrutor` for design such a framework. 

Including
- Handy block pool tools (See [BlockPools](BlockPools.ipynb) for details);
- Modules for efficiently and parallelly calculate usually used scores;
- Extendable framework for define the way blocks are updated.

## Basic Usage

To begin with, we show how to start a simple circuit construction, in which
- New blocks are added to the end of the circuit;
- The blocks score is based on the energy they can decrease by adding it to the end of the circuit;
- Energy descent gradient is calculated first and the blocks with high gradient will be selected to do a parameter optimization;
- The energy descent presented in the parameter optimization is defined to be the score of a block.

This construction can be easily carried out with the class `GreedyConstructor`. Here we start from construct the problem Hamiltonian and block pool.

In [1]:
from CircuitConstructor import GreedyConstructor
from HamiltonianGenerator import make_example_LiH
from HamiltonianGenerator.FermionTransform import jordan_wigner
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool
# Generate the problem to solve
energy_obj=make_example_LiH()
# Generate the block pool
pool=BlockPool(quasi_imaginary_evolution_rotation_pool(energy_obj.hamiltonian))

Symmetry: Coov  is used when build the molecule.


A plain constructor can be constructed with default parameters as follows.

In [ ]:
# A plain constructor with 4 processor
constructor=GreedyConstructor(energy_obj,pool) 

Constructor with various properties can be constructed by specify the parameters as follows. (Select one to run)

In [ ]:
# A 5 processor constructor with a name
from ParallelTaskRunner import TaskManager
task_manager=TaskManager(n_processor=5,task_package_size=20)
constructor=GreedyConstructor(energy_obj,pool,task_manager=task_manager,project_name="LiH") 

In [ ]:
# A constructor with specified optimizer
from ParameterOptimizere import BasinhoppingOptimizer
optimizer=BasinhoppingOptimizer(random_initial=0.01,niter=5)
constructor=GreedyConstructor(energy_obj,pool,optimizer=optimizer)

In [ ]:
# A constructor with specified initial circuit
from Blocks import BlockCircuit, HartreeFockInitBlock
bc=BlockCircuit(6)
bc.add_block(HartreeFockInitBlock([0,1]))
constructor=GreedyConstructor(energy_obj,pool,init_circuit=bc)

The construction will not start immediately after build the constructor. Use `execute_construction` to start the run. Figures of energy descent and time used, as well as log of the run will be stored in the path /mizore_result/`project_name`_`time`

In [ ]:
constructor.execute_construction()

## Extensions

### Other constuctors

We also provide a different kind of constructor `FixedDepthSweepConstructor`, which update the blocks in the circuit in a *sweep* way. 
Specifically, that is, update the blocks in the circuit of indices from `sweep_start_position` to `n_max_block-1` again and agian after constructing a circuit with `n_max_block`blocks.

Our numerical experiments show that, in this way, the number of blocks needed to achieve certain accuracy can be reduced. We also conjecture that by doing so we can avoiding local minimum in the calculation.

In [ ]:
from CircuitConstructor import FixedDepthSweepConstructor
constructor=FixedDepthSweepConstructor(energy_obj,pool,n_max_block=15,sweep_start_position=5)
constructor.execute_construction()

### More than ground state energy